In [ ]:
import pandas as pd
from breeze_connect import BreezeConnect
import mysql.connector as sqlConnector
from datetime import datetime, date, time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class DBBasic:
    con = sqlConnector.connect(host="xyz.in", user="user", passwd="*******", database="algo", port="3309")
    def __init__(self):
        pass    
    def GetTempToken(self):
        cursor = self.con.cursor()
        cursor.execute('Select SQL_NO_CACHE Token, tstamp from icicitemptoken ORDER BY ID DESC LIMIT 1;')
        for row in cursor:
            return row        

In [ ]:
# login details we got from API
api_key = "G7N34266890R6384pO940#Y4@72BU42y"
apisecret = "l~1W2017054M253KVj8L4563#7%03*F7"
db = DBBasic()

In [ ]:
breeze = BreezeConnect(api_key=api_key)

In [ ]:
def InitiateICICI():
    tempToken = db.GetTempToken()
    if tempToken is None:
        print('tempToken is None:')
        print('Login to API and rerun the program')
        return False
    ttoken = tempToken[0]
    print('ttoken:', ttoken)
    try:
        breeze.generate_session(api_secret=apisecret, session_token=ttoken)
        print('Ready to trade')
        return True
    except Exception as e:
        print(e)
        return False

In [ ]:
status = InitiateICICI()
#breeze.ws_connect()

In [ ]:
def get_historical_data(adate):
    data = breeze.get_historical_data(interval="1minute", from_date= adate + "T09:15:00.000Z",
                            to_date= adate + "T15:30:00.000Z", stock_code="NIFTY", exchange_code="NSE", product_type="cash")
    put_data = pd.DataFrame(data["Success"])
    df = put_data[['datetime', 'open', 'high', 'low', 'close']]
    df['datetime']= pd.to_datetime(df['datetime'])
    df = df.loc[df['datetime'].dt.time > time(9,14,0)]
    df = df.loc[df['datetime'].dt.time < time(15,31,0)]
    if len(df) == 0:
        print('Data not found2:', adate)
        return
    return df


In [ ]:
get_historical_data('2022-01-03').head()